In [105]:
#  -------------------------------------------


###  INPUT ###
import pandas as pd
input_data = pd.read_csv('2023_smartFarm_AI_hackathon_dataset.csv')
pd.set_option('display.max_columns', 1000)

import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
# from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler


def preprocessing(input_data):
    input_data = input_data.sort_values(["frmDist","date"])
    input_data["date"] = pd.to_datetime(input_data["date"], format='%Y%m%d')

    input_data = input_data.copy()
    input_data["frmYearWeek"] = input_data.date.dt.strftime('%Y-%W')
    input_data["frmYear"] = input_data.date.dt.strftime('%Y')
    input_data["frmWeek"] = input_data.date.dt.strftime('%W').astype(int)
    input_data = input_data.sort_values("frmYearWeek")


    tmp_df = input_data.groupby(["frmDist", 'frmYearWeek']).max().reset_index()

    # 16주 이상 농장만
    frmDist_list = (tmp_df.groupby("frmDist").frmYearWeek.count() >= 16).reset_index()

    tmp_df = pd.merge(frmDist_list.loc[frmDist_list.frmYearWeek,["frmDist"]], tmp_df, on="frmDist")

    start_point = ((tmp_df['outtrn_cumsum'] == 0)&
    (tmp_df['outtrn_cumsum'].shift(-1, fill_value=False) != 0) & 
    (tmp_df['outtrn_cumsum'].shift(1, fill_value=False) == 0))

    x_cols = ['inTp', 'inHd', 'otmsuplyqy', 'acSlrdQy', 'cunt', 
               'ph', 'outTp', 'outWs', 'daysuplyqy', 'inCo2', 'ec', 
               'frtstGrupp', 'lefstalklt', 'frtstSetCo', 'pllnLt',
               'flanGrupp', 'frtstCo', 'flanJnt', 'tcdmt', 'frmhsFclu', 'hvstGrupp',
               'hvstJnt', 'grwtLt', 'fcluHg', 'lefLt', 'flwrCo', 'hvstCo', 'lefCunt',
               'frtstJnt', 'lefBt', 'stemThck', 'frmAr', 'frmDov', 'outtrn_cumsum',
               'WaterUsage', 'WaterCost', 'FertilizerUsage', 'FertilizerCost',
               'CO2Usage', 'CO2Cost', 'MistUsageTime', 'Mist Cost',
               'HeatingEnergyUsage_cumsum'] 


    agg_dict = {i:'mean' for i in x_cols}
    agg_dict["HeatingEnergyUsage_cumsum"] = 'max'
    agg_dict["outtrn_cumsum"] = 'max'
    agg_dict["frmAr"] = 'mean'
    agg_dict["frmDist"] = 'first'
    agg_dict["frmWeek"] = 'first'
    agg_dict["date"] = 'first'
    
    result_list = []
    for i in tmp_df[start_point].index:
        sub_df = tmp_df[(i-7):(i+9)]
#         display(sub_df.iloc[12:13, :])
        if sub_df.shape[0] != 16:
#             print("NONONO", sub_df.shape[0], sub_df.frmDist.first)
            continue
#         if sub_df.frmDist.values[0] == "DBSF1673":
#             display(sub_df)
        result = sub_df.groupby('frmDist').agg(agg_dict)
        result_list.append(result)

    result_df = pd.concat(result_list, ignore_index=True)
    


    result_df["outtrn_cumsum"] = result_df["outtrn_cumsum"] /  result_df["frmAr"]
    result_df["outtrn_cumsum"] = result_df["outtrn_cumsum"] /  result_df["frmAr"]
    result_df["frmDov*frmAr"] = result_df["frmDov"] * result_df['frmAr']
    
    
    return result_df


# ... (Data preprocessing code here)
# input_data = input_data.drop(columns=['frmDist'])

input_data = preprocessing(input_data)


# Split the data into training and testing sets
X = input_data[input_data.drop(columns=['outtrn_cumsum','HeatingEnergyUsage_cumsum', "frmDist", "date"]).columns]
Y = input_data[['outtrn_cumsum','HeatingEnergyUsage_cumsum']]

X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.2, random_state=42
)

# scaler = StandardScaler()
# scaler.fit(X_train)
# X_train = scaler.transform(X_train)

# Initialize and train the LinearRegression model
model = LinearRegression()
model.fit(X_train, y_train)

# Predict 'y' values using the trained model
# X_test= scaler.transform(X_test)
y_pred = model.predict(X_test)


# Calculate RMSE between the predictions and actual 'y' values
def calculate_rmse(targets, predictions):
    """
    Calculate the Root Mean Squared Error (RMSE) between predicted and target values.

    :param predictions: Predicted values.
    :type predictions: array-like
    :param targets: Target values.
    :type targets: array-like
    :return: RMSE value.
    :rtype: float
    """
    from sklearn.metrics import mean_squared_error
    return np.sqrt(mean_squared_error(targets, predictions))


# Calculate r2_score between the predictions and actual 'y' values
def calculate_R2_score(y_test,y_pred):
    from sklearn.metrics import r2_score
    return r2_score(y_test, y_pred)


rmse = calculate_rmse(y_test, y_pred)
r2score = calculate_R2_score(y_test, y_pred)

### OUTPUT ###
print("RMSE:", rmse)
print("R2_score:", r2score)


RMSE: 0.00042555590280405557
R2_score: 0.9971887035467626


In [91]:
input_data[input_data.acSlrdQy == 0].shape

(60, 47)

In [ ]:
RMSE: 0.00035551693566831486
R2_score: 0.9983620274461023

In [95]:
pd.DataFrame({"cols":X_train.columns, "coef":model.coef_[0]}).sort_values('coef')

,cols,coef
41,frmWeek,-3.491726e-04
15,flanGrupp,-3.421828e-04
30,stemThck,-2.861321e-04
10,ec,-2.459631e-04
6,outTp,-2.272545e-04
4,cunt,-2.261187e-04
11,frtstGrupp,-1.386468e-04
19,frmhsFclu,-4.147536e-05
1,inHd,-3.919319e-05
23,fcluHg,-2.179131e-05


In [89]:
input_data.hvstGrupp

0      0.895594
1      1.135901
2      0.000000
3      1.087989
4      1.091521
         ...   
235    0.000000
236    2.076717
237    1.827168
238    0.000000
239    0.000000
Name: hvstGrupp, Length: 240, dtype: float64